In [1]:
%matplotlib widget
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import h5py 
# from BasisConvolution.util.datautils import parseFile
import os

from state import DataConfiguration
from util import processFolder, getDataLoader
from newFormatLoader import loadFrame_newFormat_v2, convertNewFormatToWCSPH, loadNewFormatState
from loader import loadState
from loader import loadBatch
from neighborhood import neighborSearch, filterNeighborhoodByKind

import warnings
from tqdm import TqdmExperimentalWarning
warnings.filterwarnings("ignore", category=TqdmExperimentalWarning)
from tqdm.autonotebook import tqdm
from state import WeaklyCompressibleSPHState, CompressibleSPHState, RigidBodyState
from batch import mergeBatch, mergeTrajectoryStates
from augment import augmentDomain, rotateState, buildRotationMatrix
from util import kernelNameToKernel
from augment import loadAugmentedBatch
from sphMath.plotting import visualizeParticles, updatePlot
import matplotlib.pyplot as plt



In [2]:

folder = '../lagrangebench/datasets/2D_TGV_2500_10kevery100'
# folder = '../lagrangebench/datasets/2D_LDC_2708_10kevery100'

configuration = DataConfiguration(
    frameDistance=1,
    frameSpacing=1,
    maxRollout=1,
    historyLength=1,
    skipInitialFrames=0,
    cutoff=0
)


In [3]:
files = os.listdir(folder)
files = [f for f in files if f.endswith('.h5')]
print(f'Found {len(files)} files in {folder}')

Found 3 files in ../lagrangebench/datasets/2D_TGV_2500_10kevery100


In [4]:
inFile = h5py.File(os.path.join(folder, files[0]), 'r')
print(f'Loading file {files[0]}')

Loading file valid.h5


In [5]:
types = np.array(inFile['00000']['particle_type'])
positions = np.array(inFile['00000']['position'])
print(types.min(), types.max())
print(positions.shape)
print(inFile['00000'].attrs.keys())

print(len(inFile.keys()))

0 0
(126, 2500, 2)
<KeysViewHDF5 []>
50


In [6]:
print(inFile['00000'].keys())
print(inFile.keys())
print(inFile.attrs.keys())

<KeysViewHDF5 ['particle_type', 'position']>
<KeysViewHDF5 ['00000', '00001', '00002', '00003', '00004', '00005', '00006', '00007', '00008', '00009', '00010', '00011', '00012', '00013', '00014', '00015', '00016', '00017', '00018', '00019', '00020', '00021', '00022', '00023', '00024', '00025', '00026', '00027', '00028', '00029', '00030', '00031', '00032', '00033', '00034', '00035', '00036', '00037', '00038', '00039', '00040', '00041', '00042', '00043', '00044', '00045', '00046', '00047', '00048', '00049']>
<KeysViewHDF5 []>


In [7]:

dtype = torch.float32
device = torch.device('cpu')

In [8]:
import json

In [9]:

metaData = folder + '/metadata.json'
parsedMetaData = json.load(open(metaData, 'r'))

In [10]:

parsedMetaData['bounds'][0]
dim = len(parsedMetaData['bounds'])
minExtent = []
maxExtent = []
for i in range(dim):
    minExtent.append(parsedMetaData['bounds'][i][0])
    maxExtent.append(parsedMetaData['bounds'][i][1])
    

print(minExtent, maxExtent)

periodic = parsedMetaData['periodic_boundary_conditions'][:dim]
print(periodic)

[0.0, 0.0] [1.0, 1.0]
[True, True]


In [11]:
from neighborhood import DomainDescription
from sphMath.operations import sph_op
from sphMath.kernels import getSPHKernelv2
from util import buildRotationMatrix
from neighborhood import coo_to_csr


domain = DomainDescription(
    min = torch.tensor(minExtent, dtype = dtype, device =device),
    max = torch.tensor(maxExtent, dtype = dtype, device =device),
    periodic = torch.tensor(periodic, dtype = torch.bool, device = device),
    dim = dim
)

print(domain)

DomainDescription(min=tensor([0., 0.]), max=tensor([1., 1.]), periodic=tensor([True, True]), dim=2)


In [12]:
trajectory = '00000'
# frame = 1

In [13]:
def computeVelocity(positions_a, positions_b, dt, domain):
    dx = positions_b - positions_a
    for i in range(domain.dim):
        if domain.periodic[i]:
            box_size = domain.max[i] - domain.min[i]
            dx_component = dx[:, i]
            dx[:, i] = ((dx_component + box_size / 2) % box_size) - box_size / 2
    return dx / dt

In [14]:
def loadFrame(inFile, trajectory, frame, parsedMetaData):
    dt = parsedMetaData['dt'] * parsedMetaData['write_every']
    positions = torch.from_numpy(inFile[trajectory]['position'][frame,:]).to(device = device, dtype=dtype)
    supports = torch.ones_like(positions[:,0], dtype=dtype, device=device) * parsedMetaData['default_connectivity_radius'] * 2

    masses = torch.ones_like(positions[:,0], dtype=dtype, device=device) * parsedMetaData['dx']**parsedMetaData['dim']

    densities = torch.ones_like(positions[:,0], dtype=dtype, device=device)

    if frame == 0:
        nextPositions = torch.from_numpy(inFile[trajectory]['position'][frame+1,:]).to(device = device, dtype=dtype)
        # speed = (nextPositions - positions) / dt
        speed = - computeVelocity(positions, nextPositions, dt, domain)
    else:
        nextPositions = torch.from_numpy(inFile[trajectory]['position'][frame-1,:]).to(device = device, dtype=dtype)
        speed = computeVelocity(positions, nextPositions, dt, domain)

    velocities = speed

    kinds = torch.from_numpy(inFile[trajectory]['particle_type'][:]).to(device = device, dtype=torch.int64).clamp(min=0, max=1)
    materials = torch.zeros_like(kinds, dtype=torch.int64, device=device)
    UIDs = torch.arange(positions.shape[0], dtype=torch.int64, device=device)

    numParticles = positions.shape[0]
    time = frame * dt
    dt = dt
    key = f'{trajectory}_{frame}'


    state = WeaklyCompressibleSPHState(
        positions=positions,
        supports=supports,
        masses=masses,
        velocities=velocities,
        densities=densities,
        kinds=kinds,
        materials=materials,
        UIDs=UIDs,
        numParticles=numParticles,
        time=time,
        dt=dt,
        key=key,
        timestep=frame,
    )

    neighborhood = neighborSearch(
        state=state,
        domain=domain,
        config=None
    )

    density = sph_op(
        state, state, domain, getSPHKernelv2('CubicSpline'), neighborhood, quantity=state.densities, supportScheme='gather', operation='density')

    state.densities = density
    filtered_csr = coo_to_csr(neighborhood)

    return state


In [15]:
from neighborhood import DomainDescription
from sphMath.operations import sph_op
from sphMath.kernels import getSPHKernelv2
from util import buildRotationMatrix
from neighborhood import coo_to_csr

for file in files:
    if 'valid' in file: 
        continue
    inFile = h5py.File(os.path.join(folder, file), 'r')
    dtype = torch.float32
    device = torch.device('cpu')
    metaData = folder + '/metadata.json'

    parsedMetaData = json.load(open(metaData, 'r'))

    parsedMetaData['bounds'][0]
    dim = len(parsedMetaData['bounds'])
    minExtent = []
    maxExtent = []
    for i in range(dim):
        minExtent.append(parsedMetaData['bounds'][i][0])
        maxExtent.append(parsedMetaData['bounds'][i][1])
        

    # print(minExtent, maxExtent)

    periodic = parsedMetaData['periodic_boundary_conditions'][:dim]
    # print(periodic)
    domain = DomainDescription(
        min = torch.tensor(minExtent, dtype = dtype, device =device),
        max = torch.tensor(maxExtent, dtype = dtype, device =device),
        periodic = torch.tensor(periodic, dtype = torch.bool, device = device),
        dim = dim
    )
    trajectories = list(inFile.keys())
    print(f'Found {len(trajectories)} trajectories in {file}')
    for trajectory in trajectories:
        # print(f'Loading trajectory {trajectory}')
        outPath = os.path.join(folder + '_converted', file.split('.')[0] + f'/{trajectory}.h5')
        # print(f'Output path: {outPath}')

        os.makedirs(os.path.dirname(outPath), exist_ok=True)

        outFile = h5py.File(outPath, 'w')
        outFile.attrs['dataType'] = 'diffSPH'
        outFile.attrs['version'] = 'LBench'

        outFile.create_group('domain')
        outFile['domain'].attrs['min'] = domain.min.cpu().numpy()
        outFile['domain'].attrs['max'] = domain.max.cpu().numpy()
        outFile['domain'].attrs['periodic'] = domain.periodic.cpu().numpy()
        # outFile['domain']
        outFile.create_group('config')

        for key in parsedMetaData.keys():
            # print(f'Setting attribute {key} to {parsedMetaData[key]}')
            outFile['config'].attrs[key] = parsedMetaData[key]
        outFile['config'].attrs['dt'] = parsedMetaData['dt'] * parsedMetaData['write_every']
        outFile['config'].attrs['num_frames'] = len(inFile[trajectory]['position'])
        outFile['config'].attrs['trajectory'] = trajectory
        outFile['config'].attrs['kernel'] = 1

        simulationDataGroup = outFile.create_group('simulationData')

        for frame in tqdm(range(inFile[trajectory]['position'].shape[0]), leave = False):
            state = loadFrame(inFile, trajectory, frame, parsedMetaData)
            # state.saveToHDF5(simulationDataGroup, frame)
            outGroup = simulationDataGroup.create_group(f'{frame:06d}')

            outGroup.attrs['numParticles'] = state.numParticles
            outGroup.attrs['time'] = state.time
            outGroup.attrs['dt'] = state.dt

            outGroup.create_dataset('positions', data=state.positions.cpu().numpy())
            outGroup.create_dataset('supports', data=state.supports.cpu().numpy())
            outGroup.create_dataset('masses', data=state.masses.cpu().numpy())
            outGroup.create_dataset('densities', data=state.densities.cpu().numpy())
            outGroup.create_dataset('velocities', data=state.velocities.cpu().numpy())
            outGroup.create_dataset('kinds', data=state.kinds.cpu().numpy())
            outGroup.create_dataset('materials', data=state.materials.cpu().numpy())
            outGroup.create_dataset('UIDs', data=state.UIDs.cpu().numpy())

            # break
            # if frame > 100:
                # break
        outFile.close()
    inFile.close()
    # break
# print(domain)

Found 100 trajectories in train.h5


  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

...

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

In [ ]:
inFile.close()
outFile.close()

In [ ]:
outPath = os.path.join(folder + '_converted', files[0].split('.')[0] + f'/{trajectory}.h5')

os.makedirs(os.path.dirname(outPath), exist_ok=True)

outFile = h5py.File(outPath, 'w')

outFile.create_group('domain')
outFile['domain'].attrs['min'] = domain.min.cpu().numpy()
outFile['domain'].attrs['max'] = domain.max.cpu().numpy()
outFile['domain'].attrs['periodic'] = domain.periodic.cpu().numpy()
# outFile['domain']
outFile.create_group('config')

for key in parsedMetaData.keys():
    # print(f'Setting attribute {key} to {parsedMetaData[key]}')
    outFile['config'].attrs[key] = parsedMetaData[key]
outFile['config'].attrs['dt'] = parsedMetaData['dt'] * parsedMetaData['write_every']
outFile['config'].attrs['num_frames'] = len(inFile[trajectory]['position'])
outFile['config'].attrs['trajectory'] = trajectory

simulationDataGroup = outFile.create_group('simulationData')

for frame in tqdm(range(inFile[trajectory]['position'].shape[0])):
    state = loadFrame(inFile, trajectory, frame, parsedMetaData)
    # state.saveToHDF5(simulationDataGroup, frame)
    outGroup = simulationDataGroup.create_group(f'{frame:06d}')

    outGroup.attrs['numParticles'] = state.numParticles
    outGroup.attrs['time'] = state.time
    outGroup.attrs['dt'] = state.dt

    outGroup.create_dataset('positions', data=state.positions.cpu().numpy())
    outGroup.create_dataset('supports', data=state.supports.cpu().numpy())
    outGroup.create_dataset('masses', data=state.masses.cpu().numpy())
    outGroup.create_dataset('densities', data=state.densities.cpu().numpy())
    outGroup.create_dataset('velocities', data=state.velocities.cpu().numpy())
    outGroup.create_dataset('kinds', data=state.kinds.cpu().numpy())
    outGroup.create_dataset('materials', data=state.materials.cpu().numpy())
    outGroup.create_dataset('UIDs', data=state.UIDs.cpu().numpy())

    # break

outFile.close()

In [ ]:
state = loadFrame(inFile, trajectory, frame, parsedMetaData)

In [ ]:
from sphMath.enums import KernelType
fig, axis = plt.subplots(1, 2, figsize=(10, 5), squeeze = False)
# filteredNeighborhood = filterNeighborhoodByKind(currentStates[0], neighborhoods[0], 'noghost')

plots = []
axes = axis.flatten()

plot = visualizeParticles(fig, axes[0],
                            particles = state,
                            domain = domain,
                            quantity = state.velocities,
                            which = 'both',
                            mapping = '.x',
                            cmap = 'viridis',
                            visualizeBoth=True,
                            kernel = KernelType.CubicSpline,
                            plotDomain = True,
                            gridVisualization=False, markerSize=2)


plot = visualizeParticles(fig, axes[1],
                            particles = state,
                            domain = domain,
                            quantity = filtered_csr.rowEntries,
                            which = 'both',
                            mapping = '.x',
                            cmap = 'viridis',
                            visualizeBoth=True,
                            kernel = KernelType.CubicSpline,
                            plotDomain = True,
                            gridVisualization=False, markerSize=2)
plots.append(plot)

fig.tight_layout()